In [ ]:
import pandas as pd
import re

base_path = "/Data/PreparedData/"

files = {
    "Embeddings": "EmbeddingRetrieverWithAnswers.csv",
    "BoW": "FrequentistRetriever_WithAnswers.csv",
    "Heurística": "HeuristicRetriever_WithAnswers.csv"
}

dataframes = {}

for strategy, filename in files.items():
    full_path = base_path + filename
    dataframes[strategy] = pd.read_csv(full_path)

def calculate_recall_at_k(row, k=10):
    """Calculates Recall@k for a single row of the DataFrame."""
    contexts = str(row["retrievedContext"]).split("||")
    gt_context = str(row["groudTruthContext"]).strip()

    for i in range(min(k, len(contexts))):
        if gt_context in contexts[i]:
            return 1
    return 0

def calculate_exact_match(row):
    return 1 if str(row['answer']).strip() == str(row['groudTruthContext']).strip() else 0

def calculate_f1_score(row):
    ground_truth_text = str(row['groudTruthContext']).lower()
    generated_answer = str(row['answer']).lower()

    ground_truth_tokens = re.findall(r'\w+', ground_truth_text)
    generated_tokens = re.findall(r'\w+', generated_answer)

    common = 0
    gt_temp = ground_truth_tokens.copy()
    for token in generated_tokens:
        if token in gt_temp:
            common += 1
            gt_temp.remove(token)

    precision = common / len(generated_tokens) if generated_tokens else 0
    recall = common / len(ground_truth_tokens) if ground_truth_tokens else 0

    if precision + recall == 0:
        return 0
    return 2 * (precision * recall) / (precision + recall)

average_metrics = {}

for strategy, df in dataframes.items():
    df['recall@10'] = df.apply(calculate_recall_at_k, axis=1)
    df['exact_match'] = df.apply(calculate_exact_match, axis=1)
    df['f1_score'] = df.apply(calculate_f1_score, axis=1)

    avg_recall = df['recall@10'].mean()
    avg_exact_match = df['exact_match'].mean()
    avg_f1_score = df['f1_score'].mean()

    average_metrics[strategy] = {
        'Recall@10': avg_recall,
        'Exact Match': avg_exact_match,
        'F1 Score': avg_f1_score
    }

print("Average Metrics for Each Strategy:")
for strategy, metrics in average_metrics.items():
    print(f"\nStrategy: {strategy}")
    for metric, value in metrics.items():
        print(f"  {metric}: {value:.4f}")


Average Metrics for Each Strategy:

Strategy: Embeddings
  Recall@10: 0.2000
  Exact Match: 0.0000
  F1 Score: 0.1659

Strategy: BoW
  Recall@10: 0.0800
  Exact Match: 0.0000
  F1 Score: 0.1548

Strategy: Heurística
  Recall@10: 0.2800
  Exact Match: 0.0000
  F1 Score: 0.1620


Recall@k: mede quantas vezes o documento correto aparece entre os k primeiros resultados recuperados.
→ Ex: se o contexto certo estiver no top 10, conta como acerto.

EM (Exact Match): verifica se a resposta gerada é idêntica à resposta esperada, sem diferenças.
→ Igualzinho, incluindo pontuação e palavras.

F1 Score: avalia o equilíbrio entre precisão (o que o modelo disse de certo) e revocação (o quanto do esperado ele acertou), com base na sobreposição de palavras.
→ Leva em conta acertos parciais.

In [ ]:
df[df.columns[4:]]

,question,answer,recall@1,exact_match,f1_score
0,How does a Non US citizen gain SEC Accredited ...,The context does not provide any information o...,0,0,0.092105
1,Buy or sell futures contracts,Buying a futures contract involves agreeing no...,1,0,0.441718
2,How to check the paypal's current exchange rate?,The context provided does not contain any info...,0,0,0.211538
3,What is the big deal about the chinese remnibi...,The Chinese Rem尼ibi trading hub opening in Tor...,0,0,0.137931
4,Is it wise to switch investment strategy frequ...,Switching investment strategies frequently can...,1,0,0.274074
...,...,...,...,...,...
95,What are good Monthly Income Funds? [Canada],The context discusses how profit depends on in...,0,0,0.086957
96,Where can I find accurate historical distribut...,To access accurate historical distribution dat...,1,0,0.198582
97,Renting or Buying an House,"Yes, there are situations where a bank does no...",0,0,0.093023
98,Buying insurance (extended warranty or guarant...,Buying insurance or extended warranty for ever...,1,0,0.143646
